# Projeto 7.1 - Tarefa: Tuning dos parãmetros

## 1. Importando bibliotecas

In [51]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch import nn, optim

In [2]:
torch.set_default_device("mps")
device = torch.device("mps")

# torch.set_default_device("cpu")
# device = torch.device("cpu")

In [3]:
print(torch.backends.mps.is_available())  # Deve retornar True
print(torch.backends.mps.is_built())  # Deve retornar True

True
True


## 2. Importando os dados

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
base = pd.read_csv('data/iris.csv')

In [6]:
previsores = base.drop(columns=['class']).copy()
classe = base['class'].copy()

In [7]:
previsores

,sepal length,sepal width,petal length,petal width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [8]:
classe

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: class, Length: 150, dtype: object

In [9]:
classe.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [10]:
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [11]:
np.unique(classe)

array([0, 1, 2])

In [12]:
previsores = np.array(previsores)
classe = np.array(classe)

In [13]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

## 3. Construindo modelo

In [14]:
class classificador_torch(nn.Module):
    def __init__(self, activation, neurons, initializer, dropout):
        super().__init__()

        self.dense0 = nn.Linear(4, neurons)
        initializer(self.dense0.weight)
        self.activation0 = activation
        self.dropout0 = nn.Dropout(dropout)

        self.dense1 = nn.Linear(neurons, neurons)
        initializer(self.dense1.weight)
        self.activation1 = activation
        self.dropout1 = nn.Dropout(dropout)

        self.dense2 = nn.Linear(neurons, 3)
        initializer(self.dense2.weight)

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout0(X)

        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dropout1(X)

        X = self.dense2(X)
        return X

## 4. Skorch

In [15]:
classificador_sklearn = NeuralNetClassifier(module=classificador_torch,
                                            optimizer__weight_decay=0.0001,
                                            train_split=False)

## 5. Tuning de parãmetros

In [16]:
params = {
    'batch_size': [10],
    'max_epochs': [50, 100],
    'optimizer': [torch.optim.Adam, torch.optim.SGD],
    'criterion': [torch.nn.CrossEntropyLoss],
    'module__activation': [F.relu, F.tanh],
    'module__neurons': [4, 8],
    'module__initializer': [torch.nn.init.uniform_, torch.nn.init.normal_],
    'module__dropout': [0.2, 0.3]
}

In [17]:
grid_search = GridSearchCV(estimator=classificador_sklearn,
                          param_grid=params,
                          scoring='accuracy',
                          cv=2,
                          error_score="raise")
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1        6.6828  0.0419
      2        3.3459  0.0261
      3        2.2968  0.0261
      4        1.5541  0.0261
      5        1.9856  0.0259
      6        1.8871  0.0257
      7        1.4681  0.0257
      8        1.5804  0.0259
      9        1.4886  0.0253
     10        1.1736  0.0231
     11        1.2068  0.0230
     12        1.2139  0.0228
     13        1.1522  0.0230
     14        1.1256  0.0230
     15        1.0993  0.0229
     16        1.0802  0.0229
     17        1.0564  0.0227
     18        1.0091  0.0231
     19        1.0065  0.0230
     20        0.9678  0.0231
     21        0.9956  0.0230
     22        0.9833  0.0240
     23        1.0383  0.0230
     24        0.9454  0.0229
     25        0.8939  0.0231
     26        0.8669  0.0230
     27        0.9176  0.0229
     28        0.8589  0.0229
     29        0.8858  0.0230
     30        0.7992  0.0261
     31        0.8380  0.0287
     32   

## 6. Resultados

In [18]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [19]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.CrossEntropyLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.tanh(input)>,
 'module__dropout': 0.2,
 'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [20]:
melhor_precisao

np.float64(0.98)

## 7. Treinando com melhores parâmetros

In [38]:
melhores_parametros

classificador = classificador_torch(
    melhores_parametros["module__activation"],
    melhores_parametros["module__neurons"],
    melhores_parametros["module__initializer"],
    melhores_parametros["module__dropout"]
)

In [39]:
base = pd.read_csv('data/iris.csv')

In [40]:
previsores = base.drop(columns=['class']).copy()
classe = base['class'].copy()

In [41]:
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [42]:
previsores_treino, previsores_teste, classe_treino, classe_teste = train_test_split(previsores,
                                                                                   classe,
                                                                                   test_size=0.25)

In [43]:
previsores_treino = torch.tensor(np.array(previsores_treino), dtype=torch.float)
classe_treino = torch.tensor(np.array(classe_treino), dtype=torch.long)

In [56]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(classificador.parameters(),
                       lr = 0.001,
                       weight_decay = 0.001)

In [57]:
dataset = torch.utils.data.TensorDataset(previsores_treino, classe_treino)

In [58]:
train_loader = torch.utils.data.DataLoader(dataset, 
                                           batch_size=melhores_parametros["batch_size"], 
                                           shuffle=True, 
                                           pin_memory=False,  # MPS não suporta pin_memory=True
                                           generator=torch.Generator(device=device))

## 4. Treinamento do modelo

In [59]:
for epoch in range(melhores_parametros["max_epochs"]):
    running_loss = 0.
    running_accuracy = 0.

    for data in train_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = classificador.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        outputs = F.softmax(outputs)
        top_p, top_class = outputs.topk(k=1, dim=1) # k=1 -> maior valor, dim=1 -> retorna a classe

        equals = top_class == labels.view(*top_class.shape) # Retorna True ou False -> comparativo entre predito e real

        running_accuracy += torch.mean(equals.type(torch.float))

        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch: {epoch + 1}, loss: {running_loss/len(train_loader)}, accuracy: {running_accuracy/len(train_loader)}")

/Users/michelarruda/miniconda3/envs/deep-learning-deAaZ-Pytorch/lib/python3.10/site-packages/torch/utils/_device.py:106: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return func(*args, **kwargs)


Epoch: 1, loss: 1.2193856835365295, accuracy: 0.40833333134651184
Epoch: 2, loss: 1.2729941109816234, accuracy: 0.30000001192092896
Epoch: 3, loss: 1.241140549381574, accuracy: 0.36666667461395264
Epoch: 4, loss: 1.1726766924063365, accuracy: 0.3499999940395355
Epoch: 5, loss: 1.196771427989006, accuracy: 0.42500004172325134
Epoch: 6, loss: 1.205255354444186, accuracy: 0.2666666507720947
Epoch: 7, loss: 1.1879200637340546, accuracy: 0.3500000238418579
Epoch: 8, loss: 1.2056687424580257, accuracy: 0.28333333134651184
Epoch: 9, loss: 1.1985087891419728, accuracy: 0.28333336114883423
Epoch: 10, loss: 1.140957271059354, accuracy: 0.32500001788139343
Epoch: 11, loss: 1.1237551867961884, accuracy: 0.4000000059604645
Epoch: 12, loss: 1.151210551460584, accuracy: 0.3333333432674408
Epoch: 13, loss: 1.124759425719579, accuracy: 0.38333335518836975
Epoch: 14, loss: 1.17241403957208, accuracy: 0.2750000059604645
Epoch: 15, loss: 1.0984563777844112, accuracy: 0.43333330750465393
Epoch: 16, loss: 1

## 8. Salvando arquivo do modelo

In [60]:
classificador.state_dict() 

OrderedDict([('dense0.weight',
              tensor([[-0.0256,  0.4804, -0.3355, -0.2151],
                      [-0.1062, -0.3054,  0.1777,  0.6535],
                      [-0.0423,  0.5416, -0.2637, -0.2568],
                      [ 0.1962,  0.4491, -0.3724, -0.5966],
                      [ 0.2849,  0.0240, -0.3763, -0.2773],
                      [ 0.0480,  0.1268,  0.0183, -0.2523],
                      [-0.0555,  0.0839, -0.0365, -0.3510],
                      [ 0.3321,  0.2315,  0.1357,  0.1032]], device='mps:0')),
             ('dense0.bias',
              tensor([ 0.0115, -0.3575, -0.0108,  0.0948, -0.1926, -0.1316, -0.2120,  0.0590],
                     device='mps:0')),
             ('dense1.weight',
              tensor([[ 0.7431, -0.2054,  0.5677,  0.1321,  0.3268,  0.1397,  0.5574,  0.1109],
                      [ 0.0491,  0.5192,  0.2364, -0.3189,  0.2894,  0.4026,  0.3942,  0.4832],
                      [-0.1763,  1.0297, -0.1517, -0.5835, -0.1137, -0.0177,  0.0949

In [61]:
torch.save(classificador.state_dict(), 'models/classificador_iris.pth')

## 9. Carregando arquivo do modelo

In [62]:
classificador_load = classificador_torch(
    melhores_parametros["module__activation"],
    melhores_parametros["module__neurons"],
    melhores_parametros["module__initializer"],
    melhores_parametros["module__dropout"]
)

In [63]:
state_dict = torch.load('models/classificador_iris.pth')
state_dict

/var/folders/rb/g2rznbp50d56w_26l6lfc72c0000gn/T/ipykernel_5473/2508085601.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('models/classificador_

OrderedDict([('dense0.weight',
              tensor([[-0.0256,  0.4804, -0.3355, -0.2151],
                      [-0.1062, -0.3054,  0.1777,  0.6535],
                      [-0.0423,  0.5416, -0.2637, -0.2568],
                      [ 0.1962,  0.4491, -0.3724, -0.5966],
                      [ 0.2849,  0.0240, -0.3763, -0.2773],
                      [ 0.0480,  0.1268,  0.0183, -0.2523],
                      [-0.0555,  0.0839, -0.0365, -0.3510],
                      [ 0.3321,  0.2315,  0.1357,  0.1032]], device='mps:0')),
             ('dense0.bias',
              tensor([ 0.0115, -0.3575, -0.0108,  0.0948, -0.1926, -0.1316, -0.2120,  0.0590],
                     device='mps:0')),
             ('dense1.weight',
              tensor([[ 0.7431, -0.2054,  0.5677,  0.1321,  0.3268,  0.1397,  0.5574,  0.1109],
                      [ 0.0491,  0.5192,  0.2364, -0.3189,  0.2894,  0.4026,  0.3942,  0.4832],
                      [-0.1763,  1.0297, -0.1517, -0.5835, -0.1137, -0.0177,  0.0949

In [64]:
classificador_load.load_state_dict(state_dict)

<All keys matched successfully>

## 10. Classificando registro

In [65]:
classificador_load.eval()

classificador_torch(
  (dense0): Linear(in_features=4, out_features=8, bias=True)
  (dropout0): Dropout(p=0.2, inplace=False)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dense2): Linear(in_features=8, out_features=3, bias=True)
)

In [66]:
novo_registro = torch.tensor([[5.1, 3.5, 1.4, 0.2]], dtype = torch.float)

In [69]:
previsao = classificador_load(novo_registro)
previsao.detach().cpu().numpy()
F.softmax(previsao)

tensor([[0.9187, 0.0712, 0.0101]], device='mps:0', grad_fn=<SoftmaxBackward0>)

In [70]:
previsoes = [np.argmax(t) for t in previsao.detach().cpu().numpy()]
previsoes

[np.int64(0)]

In [71]:
encoder.inverse_transform(previsoes)

array(['Iris-setosa'], dtype=object)